- Stitch together all tiles based on graph connections
- Calculate how many sequences are desired originals

In [9]:
import networkx as nx
from itertools import combinations
import math

G = nx.DiGraph()
G.add_nodes_from(tiles)

best_anneal = {0: 32, 1680: 30, 210: 33, 420: 34, 630: 35, 840: 30, 2100: 40, 1050: 36, 1260: 37, 1890: 39, 1470: 38}

for node1, node2 in combinations(G.nodes(), 2):
    if node1[-20:] == node2[10:30]:
        G.add_edge(node1, node2)

H = nx.DiGraph()
H.add_nodes_from(range(0,6))
H.add_edges_from([(i, i+1) for i in range(6)])
H.add_edges_from([(i, i+2) for i in range(6)])

print(list(H.edges()))


paths = []
sources = [source for source in H.nodes() if source%2 == 0]
targets = [target for target in H.nodes() if target%2 != 0]
for source in sources:
    for target in targets:
        paths.append([path for path in nx.all_simple_paths(H, source, target)])
paths
#print(list(G.edges()))

[(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (2, 4), (3, 4), (3, 5), (4, 5), (4, 6), (5, 6), (5, 7)]


[[[0, 1]],
 [[0, 1, 2, 3], [0, 1, 3], [0, 2, 3]],
 [[0, 1, 2, 3, 4, 5],
  [0, 1, 2, 3, 5],
  [0, 1, 2, 4, 5],
  [0, 1, 3, 4, 5],
  [0, 1, 3, 5],
  [0, 2, 3, 4, 5],
  [0, 2, 3, 5],
  [0, 2, 4, 5]],
 [[0, 1, 2, 3, 4, 5, 7],
  [0, 1, 2, 3, 5, 7],
  [0, 1, 2, 4, 5, 7],
  [0, 1, 3, 4, 5, 7],
  [0, 1, 3, 5, 7],
  [0, 2, 3, 4, 5, 7],
  [0, 2, 3, 5, 7],
  [0, 2, 4, 5, 7]],
 [],
 [[2, 3]],
 [[2, 3, 4, 5], [2, 3, 5], [2, 4, 5]],
 [[2, 3, 4, 5, 7], [2, 3, 5, 7], [2, 4, 5, 7]],
 [],
 [],
 [[4, 5]],
 [[4, 5, 7]],
 [],
 [],
 [],
 []]

In [2]:
tiles = ['GATTACAGATTACAGNNNNNAGCAAAAGCAGGTCAATTATATTCAGTATGGAAAGAATAAAAGAACTACGGAATCTAATGTCGCAGTCTCGCACTCGCGAGATACTGACAAAAACCACAGTGGACCATATGGCCATAATTAAGAAGTACACATCGGGGAGACAGGAAAAGAACCCGTCACTTAGGATGAAATGGATGATGGCAATGAAATATCCAATCACTGCTGACAAAAGGATAACAGAAATGGTTCCGGAGAGAAATGAACAAG', 'CACTGCTGACAAAAGGATAACAGAAATGGTTCCGGAGAGAAATGAACAAGGACAAACTCTATGGAGTAAAATGAGTGATGCTGGATCAGATAGAGTGATGGTATCACCTTTGGCTGTAACATGGTGGAATAGGAATGGACCCGTGACAAGTACGGTCCATTACCCAAAAGTGTACAAAACTTATTTCGACAAAGTCGAAAGGTTAAAACATGGAACCTTTGGCCCTGTCCATTTTAGAAATCAAGTCAAGATACGCAGAAG', 'ATGGAACCTTTGGCCCTGTCCATTTTAGAAATCAAGTCAAGATACGCAGAAGAGTAGACATAAACCCTGGTCATGCAGACCTCAGTGCCAAAGAGGCACAAGATGTAATTATGGAAGTTGTTTTTCCCAATGAAGTGGGAGCCAGAATATTAACATCAGAATCACAGCTAACAATAACTAAAGAGAAAAAAGAAGAACTCCGAGATTGCAAAATTTCTCCCTTGATGGTCGCATACATGTTAGAGAGAGAACTTGTGCGGAAA', 'AAAATTTCTCCCTTGATGGTCGCATACATGTTAGAGAGAGAACTTGTGCGGAAAACAAGATTTCTCCCAGTTGCTGGAGGAACAAGCAGCATATACATTGAAGTTTTACATTTGACTCAAGGAACGTGTTGGGAACAAATGTACACTCCAGGTGGAGAAGTGAGGAATGACGATGTTGACCAAAGCCTAATTATTGCGGCCAGGAACATAGTAAGAAGAGCCGCAGTGTCAGCAGATCCACTAGCATCTTTATTGGAGATGTGCC', 'AGTAAGAAGAGCCGCAGTGTCAGCAGATCCACTAGCATCTTTATTGGAGATGTGCCACAGCACGCAAATTGGCGGAACAAGGATGGTGGACGTTCTTAGACAGAACCCGACTGAAGAACAAGCTGTGGATATATGCAAGGCTGCAATGGGATTGAGAATCAGCTCATCCTTCAGCTTTGGTGGCTTTACATTTAAAAGAACAAGCGGGTCGTCAGTCAAAAAAGAAGAAGAGGTTCTTACAGGCAATCTCCAAACATTGAGAATAAG', 'CGTCAGTCAAAAAAGAAGAAGAGGTTCTTACAGGCAATCTCCAAACATTGAGAATAAGAGTACATGAGGGGTATGAGGAGTTCACAATGGTGGGGAAAAGAGCAACAGCTATACTGAGAAAAGCAACCAGAAGATTGGTTCAACTCATAGTGAGTGGAAGAGACGAACAGTCAATAGCCGAAGCAATAATCGTGGCCATGGTGTTTTCACAAGAAGATTGCATGATAAAAGCAGTTAGAGGTGACCTGAATTTTGTCAACAGAGCAAAT', 'CAAGAAGATTGCATGATAAAAGCAGTTAGAGGTGACCTGAATTTTGTCAACAGAGCAAATCAGCGGTTGAACCCCATGCATCAGCTTTTAAGGCATTTTCAGAAAGATGCGAAAGTACTCTTTCAAAATTGGGGAGTTGAACACATCGACAGTGTGATGGGAATGGTTGGAGTATTACCAGATATGACTCCAAGCACAGAGATGTCAATGAGAGGAATAAGAGTCAGCAAAATGGGTGTGGATGAATACTCCAGTACAGAGAGGGTGGTGG', 'GAGAGGAATAAGAGTCAGCAAAATGGGTGTGGATGAATACTCCAGTACAGAGAGGGTGGTGGTTAGCATTGATCGGTTTTTGAGAGTTCGAGACCAACGTGGGAATGTATTATTATCTCCTGAGGAGGTCAGTGAAACACAGGGAACTGAAAGACTGACAATAACTTATTCATCATCGATGATGTGGGAGATTAACGGTCCTGAGTCGGTTTTGGTCAATACCTATCAATGGATCATCAGGAATTGGGAAGCTGTCAAAATTCAATGGTCTCA', 'TTTTGGTCAATACCTATCAATGGATCATCAGGAATTGGGAAGCTGTCAAAATTCAATGGTCTCAGAATCCTGCAATGTTGTACAACAAAATGGAATTTGAACCATTTCAATCTTTAGTCCCCAAGGCCACTAGAAGCCAATACAGTGGGTTTGTCAGGACTCTATTCCAACAAATGAGAGACGTACTTGGGACATTTGACACTGCCCAGATAATAAAGCTTCTCCCTTTTGCAGCTGCTCCACCGAAGCAAAGCAGA', 'TTTGCAGCTGCTCCACCGAAGCAAAGCAGAATGCAGTTCTCTTCTTTGACTGTGAATGTGAGGGGATCAGGGATGAGAATACTTGTAAGGGGCAATTCTCCTGTATTCAACTACAACAAGACCACTAAAAGGCTAACAATTCTCGGAAAAGATGCCGGCACTTTAATTGAAGACCCAGATGAAAGCACATCCGGAGTGGAGTCCGCCGTCTTGAGAGGGTTCCTCATTATAGGTAAAGAAGACAGAAGATACGGACCA', 'CGGAGTGGAGTCCGCCGTCTTGAGAGGGTTCCTCATTATAGGTAAAGAAGACAGAAGATACGGACCAGCATTAAGCATCAATGAACTGAGTAACCTTGCAAAAGGGGAAAAGGCTAATGTGCTAATTGGGCAAGGAGACGTGGTGTTGGTAATGAAACGAAAACGGGACTCTAGTATACTTACTGACAGCCAGACAGCGACCAAAAGAATTCGGATGGCCATCAATTAATGCTGAATAGTTTAAAAACGACCTTGTTTCTACNNNNNGATTACAGATTACAG']

In [15]:
tiles

['GATTACAGATTACAGNNNNNAGCAAAAGCAGGTCAATTATATTCAGTATGGAAAGAATAAAAGAACTACGGAATCTAATGTCGCAGTCTCGCACTCGCGAGATACTGACAAAAACCACAGTGGACCATATGGCCATAATTAAGAAGTACACATCGGGGAGACAGGAAAAGAACCCGTCACTTAGGATGAAATGGATGATGGCAATGAAATATCCAATCACTGCTGACAAAAGGATAACAGAAATGGTTCCGGAGAGAAATGAACAAG',
 'CACTGCTGACAAAAGGATAACAGAAATGGTTCCGGAGAGAAATGAACAAGGACAAACTCTATGGAGTAAAATGAGTGATGCTGGATCAGATAGAGTGATGGTATCACCTTTGGCTGTAACATGGTGGAATAGGAATGGACCCGTGACAAGTACGGTCCATTACCCAAAAGTGTACAAAACTTATTTCGACAAAGTCGAAAGGTTAAAACATGGAACCTTTGGCCCTGTCCATTTTAGAAATCAAGTCAAGATACGCAGAAG',
 'ATGGAACCTTTGGCCCTGTCCATTTTAGAAATCAAGTCAAGATACGCAGAAGAGTAGACATAAACCCTGGTCATGCAGACCTCAGTGCCAAAGAGGCACAAGATGTAATTATGGAAGTTGTTTTTCCCAATGAAGTGGGAGCCAGAATATTAACATCAGAATCACAGCTAACAATAACTAAAGAGAAAAAAGAAGAACTCCGAGATTGCAAAATTTCTCCCTTGATGGTCGCATACATGTTAGAGAGAGAACTTGTGCGGAAA',
 'AAAATTTCTCCCTTGATGGTCGCATACATGTTAGAGAGAGAACTTGTGCGGAAAACAAGATTTCTCCCAGTTGCTGGAGGAACAAGCAGCATATACATTGAAGTTTTACATTTGACTCAAGGAACGTGTTGGGAACAAATGTACACTCCAGGTGGAGAAGTGAGGAATGACGATGTTGACCAAAGCCTAATT